In [1]:
import pandas as pd

""" Read the writer for every movie in the IMDB,
    from a trimmed CSV version created in the last notebook.
"""

writers = pd.read_csv('writers.list.trimmed.gz', compression='gzip', header=0, sep='\t', encoding='ISO-8859-1')

In [2]:
""" Let's just open the data up and take a peak. """

writers[writers['MOVIE_TITLE'] == "Dune"]

,WRITER,MOVIE_TITLE,YEAR
390988,"Herbert, Frank (I)",Dune,1984
390989,"Herbert, Frank (I)",Dune,NaN
554417,"Lynch, David (I)",Dune,1984


In [3]:
""" How much data do we have? """
print(len(writers))
print(len(writers['WRITER'].unique()))
print(len(writers['MOVIE_TITLE'].unique()))
print(len(writers['YEAR'].unique()))

1024178
406417
508262
135


In [4]:
""" How much broken data do we have? """

print(len(writers[writers['WRITER'].isnull()]))
print(len(writers[writers['MOVIE_TITLE'].isnull()]))
print(len(writers[writers['YEAR'].isnull()]))

0
0
30528


In [5]:
""" The above Frank Herbert example makes it look like we have some duplicates,
    that are totally artificial because of NaN years.
    It would be nice to remove these.
"""
# Remove all duplicate rows from the dataset
writers = writers.drop_duplicates()

print(len(writers))

961450


In [6]:
import numpy as np

ratings = pd.read_csv('ratings.list.trimmed.gz', compression='gzip',
                       header=0, sep='\t', encoding='ISO-8859-1',low_memory=False)

In [7]:
ratings[ratings['MOVIE_TITLE'] == 'Star Wars']

,VOTES,RANK,MOVIE_TITLE,YEAR
218798,802786,8.7,Star Wars,1977
218799,328,7.8,Star Wars,1983
218800,126,6.4,Star Wars,1988
218801,5,8.4,Star Wars,1991


In [8]:
print(len(ratings))
print(len(ratings['MOVIE_TITLE'].unique()))
ratings.drop_duplicates()
print(len(ratings))
print(len(ratings['MOVIE_TITLE'].unique()))

287119
256777
287119
256777


In [9]:
def ratings_by_person(people, ratings):
    return pd.merge(right=people, left=ratings, how='inner', on=['MOVIE_TITLE', 'YEAR'])


rated_writers = ratings_by_person(writers, ratings)

In [10]:
writers_over5k = rated_writers[rated_writers['VOTES'] > 5000]

writers_over5k = writers_over5k.sort(['RANK'], ascending=0)

print(len(writers_over5k))
writers_over5k[:5]

22318


,VOTES,RANK,MOVIE_TITLE,YEAR,WRITER
144607,23322,9.5,Hababam sinifi,1975,"Bugay, Umur"
144608,23322,9.5,Hababam sinifi,1975,"Ilgaz, Rifat"
56425,34420,9.3,CM101MMXI Fundamentals,2013,"Yilmaz, Cem"
298784,7981,9.2,RangiTaranga,2015,"Bhandari, Anup"
406567,13132,9.1,Tosun Pasa,1976,"Hikmet, Nazim"


In [11]:
writers_over25k = rated_writers[rated_writers['VOTES'] > 25000]

writers_over25k = writers_over25k.sort(['RANK'], ascending=0)

print(len(writers_over25k))
writers_over25k[:5]

9825


,VOTES,RANK,MOVIE_TITLE,YEAR,WRITER
56425,34420,9.3,CM101MMXI Fundamentals,2013,"Yilmaz, Cem"
363381,1533681,9.0,The Dark Knight,2008,"Kane, Bob (I)"
363383,1533681,9.0,The Dark Knight,2008,"Nolan, Jonathan (I)"
370296,722875,9.0,The Godfather: Part II,1974,"Puzo, Mario"
370295,722875,9.0,The Godfather: Part II,1974,"Coppola, Francis Ford"


In [12]:
# TODO: Group by writer and sort by most
print(writers_over5k[writers_over5k['WRITER'] == 'Bugay, Umur'])
print(writers_over5k[writers_over5k['WRITER'] == 'Bugay, Umur']['RANK'])
print(writers_over5k[writers_over5k['WRITER'] == 'Bugay, Umur']['RANK'].sum())
print(writers_over5k[writers_over5k['WRITER'] == 'Bugay, Umur']['RANK'].count())

        VOTES  RANK      MOVIE_TITLE  YEAR       WRITER
144607  23322   9.5   Hababam sinifi  1975  Bugay, Umur
447316  11894   8.7   Çöpçüler Krali  1977  Bugay, Umur
184221   5745   8.5  Kapicilar Krali  1976  Bugay, Umur
144607    9.5
447316    8.7
184221    8.5
Name: RANK, dtype: float64
26.7
3


In [48]:
def weighted_rank(X, v, k):
    """
    X = average for the movie (mean)
    v = number of movies
    k = minimum number movies required to be listed (currently 3)
    """
    return (v / (v + k)) * X


def rank_people(people, job, min_num):
    ranks = []
    unique_people = people[job].unique()
    for person in unique_people:
        this_person = people[people[job] == person]
        new_rank = {}
        new_rank[job] = person
        new_rank['COUNT'] = this_person['RANK'].count()
        new_rank['AVG'] = this_person['RANK'].sum() / new_rank['COUNT']
        new_rank['RANK'] = weighted_rank(new_rank['AVG'], new_rank['COUNT'], 3.0)
        ranks.append(new_rank)

    ranks = pd.DataFrame(ranks)
    return ranks.sort(['RANK'], ascending=0)

In [49]:
over_5k = rank_people(writers_over5k, 'WRITER', 3.0)

print(over_5k[:10])

           AVG  COUNT      RANK                    WRITER
53    8.000000     20  6.956522           Kurosawa, Akira
556   7.493103     29  6.790625            Hecht, Ben (I)
225   7.566667     24  6.725926             Wilder, Billy
419   8.040000     15  6.700000           Bergman, Ingmar
1077  7.079592     49  6.671154              Allen, Woody
131   7.856250     16  6.615789           Miyazaki, Hayao
528   7.147222     36  6.597436  Shakespeare, William (I)
474   7.712500     16  6.494737          Huston, John (I)
52    8.100000     12  6.480000              Oguni, Hideo
554   7.923077     13  6.437500              Buñuel, Luis


In [50]:
over_25k = rank_people(writers_over25k, 'WRITER', 3.0)

print(over_25k[:10])

          AVG  COUNT      RANK                  WRITER
690  7.237931     29  6.559375            Allen, Woody
51   8.050000     12  6.440000         Miyazaki, Hayao
401  7.444444     18  6.380952             Coen, Ethan
399  7.444444     18  6.380952              Coen, Joel
97   7.753846     13  6.300000      Tarantino, Quentin
89   8.160000     10  6.276923        Kubrick, Stanley
798  6.884615     26  6.172414        Fleming, Ian (I)
17   7.478571     14  6.158824       Lucas, George (I)
5    8.200000      9  6.150000  Nolan, Christopher (I)
6    7.009524     21  6.133333       King, Stephen (I)


In [16]:
""" This is just a really quick test to see how much difference the
    > 5k vs 25k votes cutoff makes for writers.
    The result is that it makes a big difference, on the order of
    about a third of the writers.
"""

overlap100_5k_25k = set(over_5k['WRITER'][:100].unique()).union(set(over_25k['WRITER'][:100].unique()))
print(len(overlap100_5k_25k))

overlap250_5k_25k = set(over_5k['WRITER'][:250].unique()).union(set(over_25k['WRITER'][:250].unique()))
print(len(overlap250_5k_25k))

135
342


In [17]:
""" Based on the above comparison between the over 5k and over 25k
    writers overlap, I will use the 'over 5k' collection. The
    'over 25k' collection just seems unfairly biased toward
    really popular movies, and writers who wrote a lot of movies.
"""

best_100 = over_5k[:100]
best_100[:10]

,AVG,COUNT,RANK,WRITER
53,8.000000,20,6.956522,"Kurosawa, Akira"
556,7.493103,29,6.790625,"Hecht, Ben (I)"
225,7.566667,24,6.725926,"Wilder, Billy"
419,8.040000,15,6.700000,"Bergman, Ingmar"
1077,7.079592,49,6.671154,"Allen, Woody"
131,7.856250,16,6.615789,"Miyazaki, Hayao"
528,7.147222,36,6.597436,"Shakespeare, William (I)"
474,7.712500,16,6.494737,"Huston, John (I)"
52,8.100000,12,6.480000,"Oguni, Hideo"
554,7.923077,13,6.437500,"Buñuel, Luis"


In [18]:
""" Reading the CSV dump file from IMDB that
    represents the 700 movies I have rated.
"""

# initial read of my movie ratings
my_ratings = pd.read_csv('my_ratings.csv', header=0, sep=',', encoding='ISO-8859-1',
                         usecols=['Title', 'You rated', 'Year'])
my_ratings.columns = ['MOVIE_TITLE', 'MY_RANK', 'YEAR']
print(my_ratings[:5])

# here I select my favorite movies (ratings above 7)
my_favs = my_ratings[my_ratings['MY_RANK'] > 7].sort(['MY_RANK'], ascending=0)
print(my_favs[:5])

          MOVIE_TITLE  MY_RANK  YEAR
0          Ex Machina        8  2015
1          Iron Man 3        6  2013
2       A Close Shave        7  1995
3  The Wrong Trousers        7  1993
4          Roger & Me        4  1989
                                MOVIE_TITLE  MY_RANK  YEAR
572  Futurama: Where No Fan Has Gone Before       10  1999
640                   An Inconvenient Truth       10  2006
19                                   Primer       10  2004
689                       Life Is Beautiful       10  1997
686                            Delicatessen       10  1991


In [19]:
my_fav_movies = my_favs['MOVIE_TITLE'].unique()
len(my_fav_movies)

97

In [20]:
my_fav_writers = ratings_by_person(writers_over5k, my_favs)

print(my_fav_writers[my_fav_writers['MOVIE_TITLE'] == 'Delicatessen'])
print(len(my_fav_writers))

    MOVIE_TITLE  MY_RANK  YEAR  VOTES  RANK               WRITER
1  Delicatessen       10  1991  63231   7.7       Adrien, Gilles
2  Delicatessen       10  1991  63231   7.7           Caro, Marc
3  Delicatessen       10  1991  63231   7.7  Jeunet, Jean-Pierre
231


In [21]:
my_top_writers = my_fav_writers[['WRITER', 'MY_RANK']].groupby('WRITER')\
                               .agg(['count']).sort([('MY_RANK', 'count')], ascending=0)
my_top_writers = my_top_writers[my_top_writers[('MY_RANK', 'count')] > 1]

In [22]:
print(len(my_top_writers))
my_top_writers

30


,MY_RANK
,count
WRITER,
"Gilliam, Terry",5
"Coen, Joel",4
"Coen, Ethan",4
"Chapman, Graham (I)",3
"Walsh, Fran (I)",3
"Idle, Eric",3
"Kubrick, Stanley",3
"Jackson, Peter (I)",3


In [24]:
""" Read the writer for every movie in the IMDB,
    from a trimmed CSV version created in the last notebook.
"""

directors = pd.read_csv('directors.list.trimmed.gz', compression='gzip', header=0, sep='\t', encoding='ISO-8859-1')

directors[directors['MOVIE_TITLE'] == "Dune"]

,DIRECTOR,MOVIE_TITLE,YEAR
411192,"Knoop, John (I)",Dune,1973
471676,"Lynch, David (I)",Dune,1984


In [25]:
directors[:3]

,DIRECTOR,MOVIE_TITLE,YEAR
0,"&Oumlzkul, Ahmet Salih",Ii,2013
1,"'Abd Al-Hamid, Ja'far",A Two Hour Delay,2001
2,"'Abd Al-Hamid, Ja'far",Badgeless sur la Croisette,2012


In [27]:
""" How much data do we have? """
print(len(directors))
print(len(directors['DIRECTOR'].unique()))
print(len(directors['MOVIE_TITLE'].unique()))
print(len(directors['YEAR'].unique()))

866514
318588
644799
148


In [41]:
""" Note above we have more movies and years of movies
    in IMDB with directors than writers, but we have
    fewer directors.
    This suggests that a single director will do more
    movies than a writer typically gets to do.

    So, let's test that.
"""
director_stats = directors[['DIRECTOR']].stack().value_counts()
print('\nDirectors:')
print(director_stats.describe())

writer_stats = writers[['WRITER']].stack().value_counts()
print('\nWriters:')
print(writer_stats.describe())


Directors:
count    318588.000000
mean          2.719858
std           7.512609
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max         687.000000
dtype: float64

Writers:
count    406417.000000
mean          2.365674
std           5.329415
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max         554.000000
dtype: float64


In [44]:
""" How much broken data do we have? """

print(len(directors[directors['DIRECTOR'].isnull()]))
print(len(directors[directors['MOVIE_TITLE'].isnull()]))
print(len(directors[directors['YEAR'].isnull()]))

0
0
15324


In [45]:
""" The above Frank Herbert example makes it look like we have some duplicates,
    that are totally artificial because of NaN years.
    It would be nice to remove these.
"""
# Remove all duplicate rows from the dataset
print(len(directors))
directors = directors.drop_duplicates()
print(len(directors))

866514
864726


In [46]:
""" Now let's look at the directors compared to their IMDB ratings """

rated_directors = ratings_by_person(directors, ratings)

In [47]:
""" And let's take only the movies that have had more than
    some minimum number of votes.
"""

directors_over5k = rated_directors[rated_directors['VOTES'] > 5000]
directors_over5k = directors_over5k.sort(['RANK'], ascending=0)

print(len(directors_over5k))
directors_over5k[:5]

directors_over25k = rated_directors[rated_directors['VOTES'] > 25000]
directors_over25k = directors_over25k.sort(['RANK'], ascending=0)

print(len(directors_over25k))
directors_over25k[:5]

10592
4411


,VOTES,RANK,MOVIE_TITLE,YEAR,DIRECTOR
42658,34420,9.3,CM101MMXI Fundamentals,2013,"Dundar, Murat"
281636,1558149,9.0,The Shawshank Redemption,1994,"Darabont, Frank"
267445,1066667,9.0,The Godfather,1972,"Coppola, Francis Ford"
262450,1533681,9.0,The Dark Knight,2008,"Nolan, Christopher (I)"
267450,722875,9.0,The Godfather: Part II,1974,"Coppola, Francis Ford"


In [51]:
dover_5k = rank_people(directors_over5k, 'DIRECTOR', 3.0)

In [53]:
dover_5k[:10]

,AVG,COUNT,DIRECTOR,RANK
81,7.463158,38,"Hitchcock, Alfred (I)",6.917073
33,8.187500,16,"Kurosawa, Akira",6.894737
34,7.548148,27,"Scorsese, Martin",6.793333
205,8.043750,16,"Bergman, Ingmar",6.773684
12,7.441379,29,"Spielberg, Steven",6.743750
444,7.126087,46,"Allen, Woody",6.689796
77,7.788889,18,"Wilder, Billy",6.676190
158,7.856250,16,"Wyler, William",6.615789
332,7.617647,17,"Hawks, Howard",6.475000
187,7.033333,33,"Eastwood, Clint",6.447222


In [54]:
""" Okay, now I want to go back and look at just my own ratings of movies
    And use those to find *my* favorite directors.
"""

my_fav_directors = ratings_by_person(directors_over5k, my_favs)

print(my_fav_directors[my_fav_directors['MOVIE_TITLE'] == 'Delicatessen'])
print(len(my_fav_directors))

    MOVIE_TITLE  MY_RANK  YEAR  VOTES  RANK             DIRECTOR
2  Delicatessen       10  1991  63231   7.7  Jeunet, Jean-Pierre
3  Delicatessen       10  1991  63231   7.7           Caro, Marc
101


In [55]:
my_top_directors = my_fav_directors[['DIRECTOR', 'MY_RANK']].groupby('DIRECTOR')\
                               .agg(['count']).sort([('MY_RANK', 'count')], ascending=0)
my_top_directors = my_top_directors[my_top_directors[('MY_RANK', 'count')] > 1]

print(len(my_top_directors))
print(my_top_directors)

17
                          MY_RANK
                            count
DIRECTOR                         
Gilliam, Terry                  4
Hitchcock, Alfred (I)           4
Coen, Joel                      4
Coen, Ethan                     4
Jackson, Peter (I)              3
Spielberg, Steven               3
Anderson, Wes (I)               3
Scott, Ridley                   3
Kubrick, Stanley                3
Jones, Terry (I)                3
Forman, Milos                   2
Coppola, Francis Ford           2
Kelly, Richard (II)             2
Cameron, James (I)              2
Brooks, Mel (I)                 2
Wright, Edgar                   2
Anderson, Paul Thomas (I)       2
